In [ ]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import time
import datetime as dt
import re

In [72]:
start_url = f'http://www.kicktraq.com/search/?page=1&find=in:"technology"&sort=new'

In [ ]:
req = requests.get(start_url)
soup = BeautifulSoup(req.content)

In [ ]:
project_list = soup.find('div', {'id': 'project-list'})
projects = project_list.find('div', {'class': 'projects'}).find_all('div', {'class': 'project'})
next_url = project_list.find('div', {'class': 'paging'}).find('a', {'class': 'prn'})['href']
next_url = ''.join([f'http://www.kicktraq.com/search/', next_url, f'&sort=new'])

In [ ]:
print(len(projects))

15


In [ ]:
proj = projects[0]
info = proj.find('div', {'class': 'project-infobox'})
a = info.find('h2').find()
link = f'http://www.kickstarter.com' + a['href']
name = a.text
info = proj.find('div', {'class': 'project-infobits'})
pledgilizer = info.find('div', {'class': 'project-pledgilizer'})

status = pledgilizer.find('div', {'class': 'project-pledgilizer-top'}).find().text.lower()
if status == 'closed':
    funded = pledgilizer.find('div', {'class': 'project-pledgilizer-mid'}).find().text[:-1]
    if int(funded) >= 100:
        status = 'success'
    else:
        status = 'failed'
    
funding_info = info.find('div', {'class':'project-details'}).text.split('\n\t\t\t\t\t')
moneys = funding_info[2].split(' ')
goal = moneys[3].replace(',', '')
pledged = moneys[1].replace(',', '')
backers = funding_info[1].split(' ')[1]

# With current project, dates is at index 4. With Closed project, dates is at index 3
dates = funding_info[4].split(' -> ')
start_date = dates[0].split(': ')[1]
end_date, year = dates[1].split(' (')
year = year[:-1]

In [ ]:
funding_info

['',
 'Backers: 0',
 'Funding: $0 of $5,000 (0% funded)',
 'Average daily pledges: $0',
 'Campaign Dates: January 14th -> February 15th (2021)',
 'Time left: 31 days, 20 hours, 15 minutes\n']

In [ ]:
s = f'https://www.kickstarter.com/projects/601285608/scout-the-autonomous-transatlantic-boat/?ref=kicktraq'
f = f'https://www.kickstarter.com/projects/geekazine/geekazine-special-media-new-season-of-interviews/'
status = 'failed'

kick_soup = None
kick_req = requests.get(f)
kick_soup = BeautifulSoup(kick_req.content)

In [ ]:
navigators = kick_soup.find('div', {'class': 'campaign-side-nav project-nav__links'})
comments = navigators.find('a', {'id': 'comments-emoji'}).find().text
updates = navigators.find('a', {'id': 'updates-emoji'}).find().text

faqs = navigators.find('a', {'id': 'faq-emoji'}).find()
faqs = faqs.text if faqs is not None else '0'

tiers = kick_soup.find('div', {'class': 'NS_projects__rewards_list'}).find('ol').find_all('li')
tier_count = len(tiers)
tier_min = tiers[0]
tier_max = tiers[-1]
tier_cost_min = tiers[0].find('div', {'class': 'pledge__info'}).find('span', {'class': 'money'}).text.split(' ')[1].replace(',', '')
tier_cost_max = tiers[-1].find('div', {'class': 'pledge__info'}).find('span', {'class': 'money'}).text.split(' ')[1].replace(',', '')
tier_min = int(tier_cost_min)
tier_max = int(tier_cost_max)

In [ ]:
faqs

'1'

In [ ]:
df = pd.DataFrame(columns=['Id', 'Name', 'Url', 'Goal', 'Pledged', 'Launch', 'End', 'Year', 'Comments', 'Updates', 'Faqs', 'Backers', 'Tiers', 'TierMin', 'TierMax', 'Status'])

In [73]:
next_url = start_url
pid = 1
print(df.shape)

while next_url is not None:
    url = next_url
    req = requests.get(url)

    tries = 3
    while not req.ok and tries > 0:
        print('Request to ', url, ' failed')
        time.sleep(1)
        tries -= 1
        req = requests.get(url)

    if tries <= 0:
        continue
    else:
        tries = 3

    print('Requested URL: ', url)
    
    soup = BeautifulSoup(req.content, 'html.parser')
    project_list = soup.find('div', {'id': 'project-list'})
    projects = project_list.find('div', {'class': 'projects'}).find_all('div', {'class': 'project'})
    next_url = project_list.find('div', {'class': 'paging'}).find('a', {'class': 'prn'}, text='Next >')

    if next_url is not None:
        next_url = ''.join([f'http://www.kicktraq.com/search/', next_url['href'], f'&sort=new'])

    for proj in projects:
        info = proj.find('div', {'class': 'project-infobox'})
        a = info.find('h2').find()
        link = f'http://www.kickstarter.com' + a['href']
        name = a.text
        info = proj.find('div', {'class': 'project-infobits'})
        pledgilizer = info.find('div', {'class': 'project-pledgilizer'})
        if pledgilizer is None:
            continue

        status = pledgilizer.find('div', {'class': 'project-pledgilizer-top'}).find().text.lower()
        if status != 'closed':
            continue
        else:
            funded = pledgilizer.find('div', {'class': 'project-pledgilizer-mid'}).find().text[:-1]
            if int(funded) >= 100:
                status = 'success'
            else:
                status = 'failed'

        funding_info = info.find('div', {'class':'project-details'}).text.split('\n\t\t\t\t\t')
        moneys = funding_info[2].split(' ')
        goal = moneys[3].replace(',', '')
        pledged = moneys[1].replace(',', '')
        backers = funding_info[1].split(' ')[1]

        dates = funding_info[3].split(' -> ')
        start_date = dates[0].split(': ')[1]
        end_date, year = dates[1].split(' (')
        year = year[:-1]
        
        kick_req = requests.get(link)

        tries = 3
        while not kick_req.ok and tries > 0:
            print('Request to Kickstarter ', link, ' failed')
            time.sleep(1)
            tries -= 1
            kick_req = requests.get(link)
        
        if tries <= 0:
            continue
        else:
            tries = 3
            
        kick_soup = BeautifulSoup(kick_req.content, 'html.parser')
        navigators = kick_soup.find('div', {'class': 'campaign-side-nav project-nav__links'})
        
        # Hidden projects
        if navigators is None:
            continue

        comments = navigators.find('a', {'id': 'comments-emoji'})
        comments = comments.find().text if comments is not None else 0

        updates = navigators.find('a', {'id': 'updates-emoji'})
        updates = updates.find().text if updates is not None else 0

        faqs = navigators.find('a', {'id': 'faq-emoji'}).find()
        faqs = faqs.text if faqs is not None else '0'

        tiers = kick_soup.find('div', {'class': 'NS_projects__rewards_list'}).find('ol').find_all('li', recursive=False)
        tier_count = len(tiers)
        tier_min = 0
        tier_max = 0
        if tier_count > 0:          
            tier_cost_min = tiers[0].find('div', {'class': 'pledge__info'}).find('span', {'class': 'money'}).text.split(' ')[-1].replace(',', '')
            tier_cost_max = tiers[-1].find('div', {'class': 'pledge__info'}).find('span', {'class': 'money'}).text.split(' ')[-1].replace(',', '')
        
        while not tier_cost_min.isnumeric():
            tier_cost_min = '0' if len(tier_cost_min) == 0 else tier_cost_min[1:]
        while not tier_cost_max.isnumeric():
            tier_cost_max = '0' if len(tier_cost_max) == 0 else tier_cost_max[1:]
        tier_min = int(tier_cost_min)
        tier_max = int(tier_cost_max)
        new_row = {'Id': pid, 'Name': name, 'Url': link, 'Goal': goal, 'Pledged': pledged, 
                   'Launch': start_date, 'End': end_date, 'Year': year,
                   'Comments': comments, 'Updates': updates, 'Faqs': faqs, 
                   'Backers': backers, 'Tiers': tier_count, 'TierMin': tier_min, 'TierMax': tier_max, 
                   'Status': status}
        df = df.append(new_row, ignore_index=True)
        pid += 1
        if pid % 200 == 0:
            df.to_csv('data.csv', index=None)
            print('Backed up ', pid, ' indices')

(6201, 16)
Requested URL:  http://www.kicktraq.com/search/?page=574&find=in:"technology"&sort=new
Requested URL:  http://www.kicktraq.com/search/?page=575&find=in:"technology"&sort=new
Requested URL:  http://www.kicktraq.com/search/?page=576&find=in:"technology"&sort=new
Requested URL:  http://www.kicktraq.com/search/?page=577&find=in:"technology"&sort=new
Requested URL:  http://www.kicktraq.com/search/?page=578&find=in:"technology"&sort=new
Requested URL:  http://www.kicktraq.com/search/?page=579&find=in:"technology"&sort=new
Requested URL:  http://www.kicktraq.com/search/?page=580&find=in:"technology"&sort=new
Requested URL:  http://www.kicktraq.com/search/?page=581&find=in:"technology"&sort=new
Requested URL:  http://www.kicktraq.com/search/?page=582&find=in:"technology"&sort=new
Requested URL:  http://www.kicktraq.com/search/?page=583&find=in:"technology"&sort=new
Requested URL:  http://www.kicktraq.com/search/?page=584&find=in:"technology"&sort=new
Requested URL:  http://www.kickt

In [74]:
df.to_csv('data.csv', index=None)